## Library imports

In [1]:
!pip install pretrainedmodels

     |████████████████████████████████| 58 kB 625 kB/s eta 0:00:011
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=9b2f7af0c68919c89d9ea999836b4db157a29cafa14275ebd3868927c0d4c151
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# Standard libraries
import os
from time import time
import logging
import random
from argparse import ArgumentParser
from logging import Logger
from logging.handlers import TimedRotatingFileHandler
import math
import gc
from tqdm import tqdm 

# Third party libraries
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from albumentations import (
    Compose,
    GaussianBlur,
    HorizontalFlip,
    MedianBlur,
    MotionBlur,
    Normalize,
    OneOf,
    RandomBrightness,
    RandomContrast,
    Resize,
    ShiftScaleRotate,
    VerticalFlip,
)

# torch imports
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

# pytorch lightning imports
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# extra installation
import pretrainedmodels

In [3]:
#os.environ['TORCH_HOME'] = '../input/open-source-weight-files' #setting the environment variable
#!echo $TORCH_HOME

## Constants for scripts

In [4]:
IMG_SHAPE = (600, 800, 3)
IMAGE_FOLDER = "../input/cassava-leaf-disease-classification/train_images"
SEED = 42
LOG_FOLDER = "logs"

## utils.py

In [5]:
def mkdir(path: str):
    """Create directory.

     Create directory if it is not exist, else do nothing.

     Parameters
     ----------
     path: str
        Path of your directory.

     Examples
     --------
     mkdir("data/raw/train/")
     """
    try:
        if path is None:
            pass
        else:
            os.stat(path)
    except Exception:
        os.makedirs(path)


def seed_reproducer(seed=2020):
    """Reproducer for pytorch experiment.

    Parameters
    ----------
    seed: int, optional (default = 2019)
        Radnom seed.

    Example
    -------
    seed_reproducer(seed=2019).
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.enabled = True

def init_hparams():
    parser = ArgumentParser(add_help=False)
    parser.add_argument("-backbone", "--backbone", type=str, default="se_resnext50_32x4d")
    parser.add_argument("-tbs", "--train_batch_size", type=int, default=32 * 1)
    parser.add_argument("-vbs", "--val_batch_size", type=int, default=8 * 1)
    parser.add_argument("--num_workers", type=int, default=5)
    parser.add_argument("--image_size", nargs="+", default=[256, 256])
    parser.add_argument("--seed", type=int, default=2020)
    parser.add_argument("--max_epochs", type=int, default=1)
    parser.add_argument("--gpus", nargs="+", default=[0])  # 输入1 2 3
    parser.add_argument("--precision", type=int, default=16)
    parser.add_argument("--gradient_clip_val", type=float, default=1)
    parser.add_argument("--soft_labels_filename", type=str, default="")
    parser.add_argument("--log_dir", type=str, default="logs_submit")
    try:
        hparams = parser.parse_args()
    except:
        hparams = parser.parse_args([])
    print(type(hparams.gpus), hparams.gpus)
    if len(hparams.gpus) == 1:
        hparams.gpus = [int(hparams.gpus[0])]
    else:
        hparams.gpus = [int(gpu) for gpu in hparams.gpus]

    hparams.image_size = [int(size) for size in hparams.image_size]
    return hparams


def load_data(frac=1):
    train_data = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
    test_data = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
    # Do fast experiment
    if frac < 1:
        #logger.info(f"use frac : {frac}")
        train_data = train_data.sample(frac=frac).reset_index(drop=True)
        test_data = test_data.sample(frac=frac).reset_index(drop=True)
    return train_data, test_data


def init_logger(log_name, log_dir=None):
    """日志模块
    Reference: https://juejin.im/post/5bc2bd3a5188255c94465d31
    日志器初始化
    日志模块功能:
        1. 日志同时打印到到屏幕和文件
        2. 默认保留近一周的日志文件
    日志等级:
        NOTSET（0）、DEBUG（10）、INFO（20）、WARNING（30）、ERROR（40）、CRITICAL（50）
    如果设定等级为10, 则只会打印10以上的信息

    Parameters
    ----------
    log_name : str
        日志文件名
    log_dir : str
        日志保存的目录

    Returns
    -------
    RootLogger
        Python日志实例
    """

    mkdir(log_dir)

    # 若多处定义Logger，根据log_name确保日志器的唯一性
    if log_name not in Logger.manager.loggerDict:
        logging.root.handlers.clear()
        logger = logging.getLogger(log_name)
        logger.setLevel(logging.DEBUG)

        # 定义日志信息格式
        datefmt = "%Y-%m-%d %H:%M:%S"
        format_str = "[%(asctime)s] %(filename)s[%(lineno)4s] : %(levelname)s  %(message)s"
        formatter = logging.Formatter(format_str, datefmt)

        # 日志等级INFO以上输出到屏幕
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)

        if log_dir is not None:
            # 日志等级INFO以上输出到{log_name}.log文件
            file_info_handler = TimedRotatingFileHandler(
                filename=os.path.join(log_dir, "%s.log" % log_name), when="D", backupCount=7
            )
            file_info_handler.setFormatter(formatter)
            file_info_handler.setLevel(logging.INFO)
            logger.addHandler(file_info_handler)

    logger = logging.getLogger(log_name)

    return logger


def read_image(image_path):
    """ 读取图像数据，并转换为RGB格式
        32.2 ms ± 2.34 ms -> self
        48.7 ms ± 2.24 ms -> plt.imread(image_path)
    """
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

## dataset.py

In [6]:
class PlantDataset(Dataset):
    """ Do normal training
    """

    def __init__(self, data, soft_labels_filename="", 
                transforms=None, dataset_type = 'train'):
        self.data = data
        self.transforms = transforms
        self.dataset_type = dataset_type
        if soft_labels_filename == "":
            print("soft_labels is None")
            self.soft_labels = None
        else:
            self.soft_labels = pd.read_csv(soft_labels_filename)

    def __getitem__(self, index):
        start_time = time()
        # Read image
        # solution-1: read from raw image
        image_src = f'{IMAGE_FOLDER}/{self.data.loc[index, "image_id"]}'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #print(type(image))        
        #print(image.shape, labels.shape)

        # Convert if not the right shape
        #if image.shape != IMG_SHAPE:
        #    image = image.transpose(1, 0, 2)

        # Do data augmentation
        if self.transforms is not None:
            image = self.transforms(image=image)["image"].transpose(2, 0, 1)

        # Soft label
        #if self.soft_labels is not None:
        #    label = torch.FloatTensor(
        #        (self.data.iloc[index, 1:].values * 0.7).astype(np.float)
        #        + (self.soft_labels.iloc[index, 1:].values * 0.3).astype(np.float)
        #    )
        #else:
        #    label = torch.FloatTensor(self.data.iloc[index, 1:].values.astype(np.int64))
        
        if self.dataset_type == 'train':
            label = self.data.loc[index, ['cls0', 'cls1', 'cls2', 'cls3', 'cls4']].values
            label = torch.from_numpy(label.astype(np.int8))
            #labels = labels.unsqueeze(-1)
        else:
            labels = torch.Tensor(1)        

        return image, label, time() - start_time

    def __len__(self):
        return len(self.data)


def generate_transforms(image_size):

    train_transform = Compose(
        [
            Resize(height=image_size[0], width=image_size[1]),
            OneOf([RandomBrightness(limit=0.1, p=1), RandomContrast(limit=0.1, p=1)]),
            OneOf([MotionBlur(blur_limit=3), MedianBlur(blur_limit=3), GaussianBlur(blur_limit=3)], p=0.5),
            VerticalFlip(p=0.5),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(
                shift_limit=0.2,
                scale_limit=0.2,
                rotate_limit=20,
                interpolation=cv2.INTER_LINEAR,
                border_mode=cv2.BORDER_REFLECT_101,
                p=1,
            ),
            Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
        ]
    )

    val_transform = Compose(
        [
            Resize(height=image_size[0], width=image_size[1]),
            Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
        ]
    )

    return {"train_transforms": train_transform, "val_transforms": val_transform}


def generate_dataloaders(hparams, train_data, val_data, transforms):
    train_dataset = PlantDataset(
        data=train_data, transforms=transforms["train_transforms"], soft_labels_filename=hparams.soft_labels_filename
    )
    val_dataset = PlantDataset(
        data=val_data, transforms=transforms["val_transforms"], soft_labels_filename=hparams.soft_labels_filename
    )
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=hparams.train_batch_size,
        shuffle=True,
        num_workers=hparams.num_workers,
        pin_memory=False,
        drop_last=False,
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=hparams.val_batch_size,
        shuffle=False,
        num_workers=hparams.num_workers,
        pin_memory=False)

    return train_dataloader, val_dataloader

## loss_function.py

In [7]:
class CrossEntropyLossOneHot(nn.Module):
    def __init__(self):
        super(CrossEntropyLossOneHot, self).__init__()
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, preds, labels):
        #print(preds.shape, labels.shape)
        return torch.mean(torch.sum(-labels * self.log_softmax(preds), -1))

In [8]:
#check_point = torch.load('../input/open-source-weight-files/se_resnext50_32x4d-a260b3a4.pth')
#se_resnext_50_imagenet = pretrainedmodels.se_resnext50_32x4d(num_classes=1000,pretrained=None)
#se_resnext_50_imagenet.load_state_dict(check_point)

## models.py

In [9]:
def l2_norm(input, axis=1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)
    return output


class BinaryHead(nn.Module):
    def __init__(self, num_class=5, emb_size=2048, s=16.0):
        super(BinaryHead, self).__init__()
        self.s = s
        self.fc = nn.Sequential(nn.Linear(emb_size, num_class))

    def forward(self, fea):
        fea = l2_norm(fea)
        logit = self.fc(fea) * self.s
        return logit


class se_resnext50_32x4d(nn.Module):
    def __init__(self):
        super(se_resnext50_32x4d, self).__init__()
        check_point = torch.load('../input/open-source-weight-files/se_resnext50_32x4d-a260b3a4.pth')
        se_resnext_50_imagenet = pretrainedmodels.se_resnext50_32x4d(num_classes=1000,pretrained=None)
        se_resnext_50_imagenet.load_state_dict(check_point)
        self.model_ft = nn.Sequential(*list(se_resnext_50_imagenet.children())[:-2])
        
        #self.model_ft = nn.Sequential(
        #    *list(pretrainedmodels.__dict__["se_resnext50_32x4d"](num_classes=1000, pretrained="imagenet",
        #            weight_path='../input/open-source-weight-files/se_resnext50_32x4d-a260b3a4.pth').children())[:-2]
        #)
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.model_ft.last_linear = None
        self.fea_bn = nn.BatchNorm1d(2048)
        self.fea_bn.bias.requires_grad_(False)
        self.binary_head = BinaryHead(5, emb_size=2048, s=1)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):

        img_feature = self.model_ft(x)
        img_feature = self.avg_pool(img_feature)
        img_feature = img_feature.view(img_feature.size(0), -1)
        fea = self.fea_bn(img_feature)
        # fea = self.dropout(fea)
        output = self.binary_head(fea)

        return output

In [10]:
#temp = se_resnext50_32x4d()
#print(temp)

## train.py

In [11]:
class CoolSystem(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        # 让每次模型初始化一致, 不让只要中间有再次初始化的情况, 结果立马跑偏
        seed_reproducer(self.hparams.seed)
        self.model = se_resnext50_32x4d()
        self.criterion = CrossEntropyLossOneHot()
        #self.logger_kun = init_logger("kun_in", hparams.log_dir)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
        #self.scheduler = WarmRestart(self.optimizer, T_max=10, T_mult=1, eta_min=1e-5)
        self.scheduler = lr_scheduler.CosineAnnealingWarmRestarts(self.optimizer, T_0=10, T_mult=1, eta_min=1e-5)
        return [self.optimizer], [self.scheduler]

    def training_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels,_ = batch
        scores = self(images)
        loss = self.criterion(scores, labels)
        #data_load_time = torch.sum(data_load_time)
        # self.logger_kun.info(f"loss : {loss.item()}")
        # ! can only return scalar tensor in training_step
        # must return key -> loss
        # optional return key -> progress_bar optional (MUST ALL BE TENSORS)
        # optional return key -> log optional (MUST ALL BE TENSORS)
        
        
        # identifying number of correct predections in a given batch
        #correct=scores.argmax(dim=1).eq(labels).sum().item()
        # identifying total number of labels in a given batch
        #total=len(labels)

        return {
            "loss": loss,
            #"data_load_time": data_load_time,
            #"batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(data_load_time.device),
            "scores": scores, "labels": labels,
        }

    def training_epoch_end(self, outputs):
        # outputs is the return of training_step
        train_loss_mean = torch.stack([output["loss"] for output in outputs]).mean()
        #self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        #self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # calculating correect and total predictions
        # correct=sum([x["correct"] for  x in outputs])
        # total=sum([x["total"] for  x in outputs])
        
        # compute accuracy
        scores_all = torch.argmax(torch.cat([output["scores"] for output in outputs]), dim=1).cpu().detach().numpy()#.astype(int)
        labels_all = torch.argmax(torch.cat([output["labels"] for output in outputs]), dim=1).cpu().detach().numpy()
        train_acc_score = accuracy_score(labels_all, scores_all)

        # logging using tensorboard logger
        self.logger.experiment.add_scalar("Loss/Train", train_loss_mean, self.current_epoch)
        self.logger.experiment.add_scalar("Accuracy_score/Train", train_acc_score, self.current_epoch)
        print(f"Epoch : {self.current_epoch}, Fold : {self.hparams.fold_i}, train_loss : {train_loss_mean}, train_acc_score: {train_acc_score}")

    def validation_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels,_ = batch
        #data_load_time = torch.sum(data_load_time)
        scores = self(images)
        loss = self.criterion(scores, labels)

        # must return key -> val_loss
        return {
            "val_loss": loss, "scores": scores, "labels": labels,
            #"data_load_time": data_load_time,
            #"batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(data_load_time.device),
        }

    def validation_epoch_end(self, outputs):
        # compute loss
        val_loss_mean = torch.stack([output["val_loss"] for output in outputs]).mean()
        #self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        #self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # compute accuracy
        scores_all = torch.argmax(torch.cat([output["scores"] for output in outputs]), dim=1).cpu().detach().numpy()#.astype(int)
        labels_all = torch.argmax(torch.cat([output["labels"] for output in outputs]), dim=1).cpu().detach().numpy()
                
        #print('scores_all[0]', scores_all[0], scores_all.shape, type(scores_all))
        #print('labels_all[0]', labels_all[0], labels_all.shape, type(labels_all))
        val_acc_score = accuracy_score(labels_all, scores_all)
        
        # logging using tensorboard logger
        self.logger.experiment.add_scalar("Loss/Validation", val_loss_mean, self.current_epoch)
        self.logger.experiment.add_scalar("Accuracy_score/Validation", val_acc_score, self.current_epoch)

        # terminal logs
        #self.logger_kun.info(
        #    f"{self.hparams.fold_i}-{self.current_epoch} | "
        #    f"lr : {self.scheduler.get_lr()[0]:.6f} | "
        #    f"val_loss : {val_loss_mean:.4f} | "
        #    f"val_roc_auc : {val_roc_auc:.4f} | "
        #    f"data_load_times : {self.data_load_times:.2f} | "
        #    f"batch_run_times : {self.batch_run_times:.2f}"
        #)
        # f"data_load_times : {self.data_load_times:.2f} | "
        # f"batch_run_times : {self.batch_run_times:.2f}"
        # must return key -> val_loss
        
        # for EarlyStopping callback
        self.log('val_acc_score', val_acc_score)
        print(f"Epoch : {self.current_epoch}, Fold : {self.hparams.fold_i}, val_loss : {val_loss_mean}, val_acc_score: {val_acc_score}")
        #return {"val_loss": val_loss_mean, "val_roc_auc": val_roc_auc}

In [12]:
!rm -rf kaggle/working/*

In [13]:
# Make experiment reproducible
seed_reproducer(42)

# Init Hyperparameters
hparams = init_hparams()

# init logger
#logger = init_logger("kun_out", log_dir=hparams.log_dir)

# Load data
train_df, test_df = load_data(frac=1)
train_df[['cls0', 'cls1', 'cls2', 'cls3', 'cls4']] = train_labels = pd.get_dummies(train_df.iloc[:, 1])
train_labels = train_df.iloc[:, 1].values
print(train_df.shape, train_labels.shape)

# Generate transforms
transforms = generate_transforms(hparams.image_size)

# Do cross validation
valid_roc_auc_scores = []
#folds = KFold(n_splits=5, shuffle=True, random_state=hparams.seed)
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
oof_preds = np.zeros((train_df.shape[0],))

<class 'list'> [0]
(21397, 7) (21397,)


usage: ipykernel_launcher.py [-backbone BACKBONE] [-tbs TRAIN_BATCH_SIZE]
                             [-vbs VAL_BATCH_SIZE] [--num_workers NUM_WORKERS]
                             [--image_size IMAGE_SIZE [IMAGE_SIZE ...]]
                             [--seed SEED] [--max_epochs MAX_EPOCHS]
                             [--gpus GPUS [GPUS ...]] [--precision PRECISION]
                             [--gradient_clip_val GRADIENT_CLIP_VAL]
                             [--soft_labels_filename SOFT_LABELS_FILENAME]
                             [--log_dir LOG_DIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-59d623e6-d9bf-4a5b-86eb-43647aa0063a.json
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [14]:
#temp_dataset = PlantDataset(train_df)
#temp = temp_dataset[0]
#temp[1]

In [15]:
for fold_i, (train_index, val_index) in enumerate(folds.split(train_df, train_labels)):
    hparams.fold_i = fold_i
    train_data = train_df.iloc[train_index, :].reset_index(drop=True)
    val_data = train_df.iloc[val_index, :].reset_index(drop=True)
    train_dataloader, val_dataloader = generate_dataloaders(hparams, train_data, val_data, transforms)
    
    # logger function
    logger = TensorBoardLogger(LOG_FOLDER + str(fold_i), name="se_resnext50_v1")
    
    # Define callbacks
    checkpoint_callback = ModelCheckpoint(monitor="val_acc_score",
        filepath=os.path.join(LOG_FOLDER, f"fold={fold_i}" + "-{epoch}-{val_loss:.4f}-{val_acc_score:.4f}"),
                                          save_top_k =1
    )
    
    #early_stop_callback = EarlyStopping(monitor="val_roc_auc", patience=3, mode="max", verbose=True)
    
    # Instance Model, Trainer and train model
    model = CoolSystem(hparams)
    trainer = pl.Trainer(
        gpus=hparams.gpus,
        min_epochs=5,
        max_epochs=hparams.max_epochs,
        #early_stop_callback=early_stop_callback,
        #callbacks=[early_stop_callback],
        checkpoint_callback=checkpoint_callback,
        progress_bar_refresh_rate=0,
        precision=hparams.precision,
        num_sanity_val_steps=0,
        profiler=False,
        weights_summary=None,
        #use_dp=True,
        gradient_clip_val=hparams.gradient_clip_val, 
        logger=logger
    )
    trainer.fit(model, train_dataloader, val_dataloader)

    #valid_roc_auc_scores.append(round(checkpoint_callback.best, 4))
    #logger.info(valid_roc_auc_scores)

    del model
    gc.collect()
    torch.cuda.empty_cache()

soft_labels is None
soft_labels is None


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Epoch : 0, Fold : 0, val_loss : 0.85923832654953, val_acc_score: 0.707168894289186
Epoch : 0, Fold : 0, train_loss : 0.9188690781593323, train_acc_score: 0.6920919798093101
soft_labels is None
soft_labels is None


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /kaggle/working/logs exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Epoch : 0, Fold : 1, val_loss : 0.7006984353065491, val_acc_score: 0.7564030659936437
Epoch : 0, Fold : 1, train_loss : 0.9351182579994202, train_acc_score: 0.6690344892045985


In [16]:
#del model
#gc.collect()
#torch.cuda.empty_cache()